In [1]:
"""plan 
    allow comments
    prevent lingering options
    fix borderWidth on items
""";

In [90]:
graphString="""id a
	label a
	linkto
id b
nodes
	color pink
	color border red"""
graphString

'id a\n\tlabel a\n\tlinkto\nid b\nnodes\n\tcolor pink\n\tcolor border red'

In [73]:
graphString="""id a
	label a
	linkto  b
id b
	linkto
	label b
nodes
	color pink
    color red
    color border red
	borderWidth 10
"""

9

In [98]:
def getChunks(graphString=graphString):
    global lines
    """each chunk is a string that needs to be converted into a record"""
    lines = graphString.split('\n')
    withBreaks = []
    #print('lines', lines)
    for line in lines:
        while line.startswith('    '):
            line=line.replace('    ','\t')
        if not line.startswith('\t'):
            withBreaks.append('@@' + line)
        else:
            withBreaks.append(line)
    rejoined = '\n'.join(withBreaks)
    #print('rejoined', rejoined)
    ret=[ _ for _  in rejoined.split('@@') if _.strip()]
    return ret
getChunks()

['id a\n\tlabel a\n\tlinkto\n',
 'id b\n',
 'nodes\n\tcolor pink\n\tcolor border red']

In [ ]:
['id a\n\tlabel a\n\tlinkto  b\n',
 'id b\n\tlinkto\n\tlabel b\n',
 'nodes\n\tcolor pink\n\tcolor red\n\tborderWidth 10\n']

In [89]:
keywords = """"id label url title linkto color shape
font nodes edges x y layout physics hierarchical border
borderWidth background opacity hidden""".split()

def getRecords(graphString):
    """each record is a list of phrases taht will become a key value pair
    complexity comes from fact that labels can be multi-line"""
    chunks = getChunks(graphString)
    records = []
    for chunk in chunks:
        for keyword in keywords: #we are now assuming indents
            chunk=chunk.replace('\n\t'+ keyword,'BREAK'+ keyword) #keywords must be at beginning
        lines=chunk.split('BREAK')
        records.append([line.strip() for line in lines if line.strip()])
    return records #used by getOptions and getNodes

getRecords(graphString)

[['id a', 'label a', 'linkto  b'],
 ['id b', 'linkto', 'label b'],
 ['nodes', 'color pink', 'color red', 'color border red', 'borderWidth 10']]

In [144]:
for line in getChunks():
    print(line)
    print('--')
    if 

id a
	label a
	linkto

--
id b

--
nodes
	color pink
	color border red
--


In [125]:
####new approach
linesWithOptions=[]
for line in getChunks():
    render(line)
  

{id:
{id:
{nodes
	color:


In [11]:
def xxparseOptions(graphString=graphString):
    """Create nested dictionaries as required by visJS.  
    At end, optionProcessing is complete, but nodes and edges need more massaging.
    """
    def fixV(s):
        """deal with non-string values"""
        if s in ['True', 'true']: return True
        if s in ['False', 'false']: return False
        try: # to convert to a number
            return(int(s))
        except:
            return s #then just return as is
    

    records = getRecords(graphString)
    newOpts = records
    options={}
    for newOpt in newOpts:
        kind = newOpt[0]
        options[kind]= {}
        for opt in newOpt[1:]:
            if len(opt.split())>1:
                k,vs = opt.split()[0], opt.split()[1:]
                ###################
                if k=='linkto':  #don't turn into dictionaries
                    options[kind][k]=' '.join(vs) #leave linktos as space delimited 'ONE TWO THREE'
                    break

                print('lenVs',len(vs), vs)

                if len(vs)==1:
                    v=vs[0]
                    options[kind][k] = fixV(v)

                if len(vs)==2:
                    k2, v = vs
                    if k not in options[kind]: #make sure we have the dict created
                        try:
                            options[kind][k]=dict()
                        except:
                            print('error')

                
                    options[kind][k][k2] = fixV(v)

                if len(vs)==3: #will fail beyond this
                    k2, k3, v = vs
                    if k not in options[kind]:
                        options[kind][k]=dict()

                    if k2 not in options[kind][k]:
                        try:
                            options[kind][k][k2] = dict()
                        except TypeError:
                            print('ERROR')

                    options[kind][k][k2][k3] = fixV(v)
    return options
xxparseOptions()

lenVs 1 ['a']
lenVs 1 ['b']
lenVs 1 ['pink']
lenVs 1 ['red']
lenVs 1 ['10']


{'id a': {'label': 'a', 'linkto': 'b'},
 'id b': {'label': 'b'},
 'nodes': {'color': 'red', 'borderWidth': 10}}

In [88]:
def parseOptions(graphString=graphString):
    records = getRecords(graphString)
    #print(records)
    entries=[]
    for record in records:
        kind = record.pop(0)
        for line in record:
            words = line.split()
            value = words.pop()
            entries.append(dict(kind=kind, words=words, value=value))
    return entries
    
parseOptions()

[{'kind': 'id a', 'words': ['label'], 'value': 'a'},
 {'kind': 'id a', 'words': ['linkto'], 'value': 'b'},
 {'kind': 'id b', 'words': [], 'value': 'linkto'},
 {'kind': 'id b', 'words': ['label'], 'value': 'b'},
 {'kind': 'nodes', 'words': ['color'], 'value': 'pink'},
 {'kind': 'nodes', 'words': ['color'], 'value': 'red'},
 {'kind': 'nodes', 'words': ['color', 'border'], 'value': 'red'},
 {'kind': 'nodes', 'words': ['borderWidth'], 'value': '10'}]

In [85]:
POs = parseOptions()
nodes=[]
edges=[]
option=[]
for PO in POs:
    if PO['kind'].startswith('id'):
        node=dict(id = PO['kind'].split('id ')[1])
    else:
        node=dict()
    nodes.append(node)
        
nodes
    

[{'id': 'a'}, {'id': 'a'}, {'id': 'b'}, {'id': 'b'}, {}, {}, {}, {}]

In [145]:
def nodesEdgesOptions(graphString=graphString):
    """create a dictionary that has nodes, edges and options formatted as required by visjs
    """
    entries=[]
    PO = parseOptions(graphString)
    entries= dict(nodes=[], edges=[], options={})
    for k,v in PO.items():
        if k.startswith('id'): #create a node entry
            id = k.split('id')[1].strip()
            node = {'id': id}
            for k2,v2 in v.items():
                node[k2]=v2
            entries['nodes'].append(node)
        else:
            entries['options'][k]=v  #create an option entry (we're expecting nodes, layout, physics, edges...maybe interaction )

    for node in entries['nodes']: #create edges entries
        if 'linkto' in node.keys():
            for linkto in node['linkto'].split(' '):
                entries['edges'].append( {'from':node['id'], 'to':linkto} )

    return entries

nodesEdgesOptions()['options']

AttributeError: 'list' object has no attribute 'items'

In [146]:
import subprocess
if subprocess.run.__doc__:
    cmd = 'jupyter nbconvert --to python parseLeft.ipynb'
    subprocess.run(cmd.split(' '))
